# BB84 — Alice Role

This notebook creates the `alice_notebook.py` server script and shows how to run it.

**Pre-requisites**:

- Python 3.9+
- Install: `pip install qiskit qiskit-ibm-runtime flask requests numpy cryptography`

Make sure QiskitRuntimeService credentials are saved on machines that will call IBM backends.

In [ ]:
# Write the role script to disk
script = r"""
# alice.py
from flask import Flask, request, jsonify
import requests, time, numpy as np
app = Flask(__name__)

# CONFIG - change these to actual host:port (use http://<IP>:<port>)
EVE_HOST = "http://<EVE_IP>:192.168.0.107"
BOB_HOST = "http://<BOB_IP>:192.168.0.107"

N = 64
alice_bits = None
alice_bases = None
final_key = None

@app.route("/start_protocol", methods=["POST"])
def start_protocol():
    global alice_bits, alice_bases
    np.random.seed(int(time.time()) % 100000)
    alice_bits = np.random.randint(2, size=N).tolist()
    alice_bases = np.random.randint(2, size=N).tolist()
    payload = [{"idx": i, "bit": int(alice_bits[i]), "basis": int(alice_bases[i])} for i in range(N)]
    r = requests.post(f"{EVE_HOST}/receive_from_alice", json={"preps": payload})
    return jsonify({"status": "sent to Eve", "eve_reply": r.text})

@app.route("/announce_bases", methods=["GET"])
def announce_bases():
    global alice_bases
    if alice_bases is None:
        return jsonify({"error": "no session"}), 400
    r = requests.post(f"{BOB_HOST}/receive_alice_bases", json={"bases": alice_bases})
    return jsonify({"status": "bases_sent_to_bob", "bob_reply": r.text})

@app.route("/set_final_key", methods=["POST"])
def set_final_key():
    global final_key
    final_key = request.json.get("key_bits", None)
    if final_key is None:
        return jsonify({"error":"no key provided"}), 400
    return jsonify({"status":"final key set", "length": len(final_key)})

@app.route("/send_message", methods=["POST"])
def send_message():
    global final_key
    if final_key is None:
        return jsonify({"error": "no final key available"}), 400
    plaintext = request.json.get("message", "")
    pb = plaintext.encode('utf-8')
    key_bits = list(final_key)
    while len(key_bits) % 8 != 0:
        key_bits.append(0)
    key_bytes = bytes([int(''.join(str(b) for b in key_bits[i:i+8]), 2) for i in range(0, len(key_bits), 8)])
    key_bytes = key_bytes[:len(pb)]
    ciphertext = bytes([a ^ b for a, b in zip(pb, key_bytes)])
    r = requests.post(f"{BOB_HOST}/receive_ciphertext", json={"ciphertext": list(ciphertext)})
    return jsonify({"status": "sent", "bob_reply": r.text})

if __name__ == "__main__":
    print("Alice server running on port 5000")
    app.run(host="0.0.0.0", port=5000)

"""
path = '/mnt/data/alice_notebook.py'
with open(path, 'w', encoding='utf-8') as f:
    f.write(script)
print('Wrote', path)

print('\nEdit the host addresses inside the script file, then run in a terminal:')
print('python', path)


Edit EVE_HOST and BOB_HOST in the script file; then run it.